# HW05-Deployment

## Question 3

Let's use the model!

* Write a script for loading the pipeline with pickle
* Score this record:

```json
{
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
```

What's the probability that this lead will convert? 

* 0.333
* 0.533
* 0.733
* 0.933

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum pipeline_v1.bin
7d17d2e4dfbaf1e408e1a62e6e880d49 *pipeline_v1.bin
```

In [ ]:
import pickle

def predict(record, model):
    x_customer = 

    with open('pipeline_v1.bin', 'rb') as f_in:
        model = pickle.load(f_in)

    record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
    }

In [7]:
import pickle
import numpy as np

# model
model = "pipeline_v1.bin"


def predict_conversion_proba(record, pipeline):

    # The pipeline expects a list of dictionaries, even for a single record
    X_record = [record]
    y_pred_proba = pipeline.predict_proba(X_record)[:, 1]

    # Return the single probability value
    return y_pred_proba[0]


with open(model, "rb") as f_in:
    # Load the complete scikit-learn pipeline (DictVectorizer + LogisticRegression)
    pipeline = pickle.load(f_in)

# Score the client record
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0,
}

# Calculate the probability
probability = predict_conversion_proba(record, pipeline)
print(f"The probability that this lead will convert is: {probability}")

The probability that this lead will convert is: 0.5336072702798061


## Question 4

Now let's serve this model as a web service

* Install FastAPI
* Write FastAPI code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription?

* 0.334
* 0.534
* 0.734
* 0.934


--- Prediction Result ---
{'conversion_probability': 0.5340417283801275}

## Docker

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `agrigorev/zoomcamp-model:2025`. 
You'll need to use it (see Question 5 for an example).

This image is based on `3.13.5-slim-bookworm` and has
a pipeline with logistic regression (a different one)
as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.13.5-slim-bookworm
WORKDIR /code
COPY pipeline_v2.bin .
```

We already built it and then pushed it to [`agrigorev/zoomcamp-model:2025`](https://hub.docker.com/r/agrigorev/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.


## Question 5

Download the base image `agrigorev/zoomcamp-model:2025`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 45 MB
* 121 MB
* 245 MB
* 330 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [9]:
!docker pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model
59e22667830b: Pulling fs layer
19f81082f6f0: Pulling fs layer
799df5177ae2: Pulling fs layer
66cd357cf7a1: Pulling fs layer
760c2b9cc6ee: Pulling fs layer
816e9c0578b5: Pulling fs layer
816e9c0578b5: Download complete
19f81082f6f0: Download complete
66cd357cf7a1: Download complete
760c2b9cc6ee: Download complete
799df5177ae2: Download complete
59e22667830b: Download complete
59e22667830b: Pull complete
19f81082f6f0: Pull complete
816e9c0578b5: Pull complete
66cd357cf7a1: Pull complete
760c2b9cc6ee: Pull complete
799df5177ae2: Pull complete
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [10]:
!docker images

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      14d79fde0bbf   6 days ago   181MB


### choose the most close size 121MB

## Dockerfile

Now create your own `Dockerfile` based on the image we prepared.

It should start like that:

```docker
FROM agrigorev/zoomcamp-model:2025
# add your stuff here
```

Now complete it:

* Install all the dependencies from pyproject.toml
* Copy your FastAPI script
* Run it with uvicorn 

After that, you can build your docker image.

## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()
```

What's the probability that this lead will convert?

* 0.39
* 0.59
* 0.79
* 0.99


## Publishing to Docker hub

This is just for reference, this is how we published an image to Docker hub.

`Dockerfile_base`: 

```dockerfile
FROM python:3.13.5-slim-bookworm
WORKDIR /code
COPY pipeline_v2.bin .
```

Publishing:

```bash
docker build -t mlzoomcamp2025_hw5 -f Dockerfile_base .
docker tag mlzoomcamp2025_hw5:latest agrigorev/zoomcamp-model:2025
docker push agrigorev/zoomcamp-model:2025
```
